# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 20902017


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/32 [00:00<?, ?it/s]

Rendering models:   6%|▋         | 2/32 [00:06<01:37,  3.26s/it]

Rendering models:   9%|▉         | 3/32 [00:08<01:20,  2.77s/it]

Rendering models:  12%|█▎        | 4/32 [00:08<00:59,  2.11s/it]

Rendering models:  16%|█▌        | 5/32 [00:09<00:45,  1.67s/it]

...

Rendering models:  88%|████████▊ | 28/32 [00:22<00:02,  1.79it/s]

Rendering models:  91%|█████████ | 29/32 [00:23<00:01,  1.72it/s]

Rendering models:  94%|█████████▍| 30/32 [00:23<00:01,  1.65it/s]

Rendering models:  97%|█████████▋| 31/32 [00:24<00:00,  1.51it/s]

equidad1                              0.054411
klmasters                             0.000115
BeNjEsSePh                            0.000099
Khaazel                               0.000093
kayleebug2017                         0.000376
bronzesword                           0.000147
JessieDawn                            0.000805
not-logged-in-ea8ed61a7274cfb6a43b    0.002188
Linda_J._Berkel                       0.000164
not-logged-in-0b0736c8991a7cc2017a    0.000349
StephAChamber                         0.000154
ImArobot                              0.003863
browny811                             0.000268
MeepTastic                            0.000204
addison.brooks                        0.003727
41Letters                             0.000159
pangeli5                              0.000144
not-logged-in-9f30f411df9d28d2c541    0.003739
mjcb4                                 0.001406
not-logged-in-05a6abd41fcd16a89d67    0.000743
Mr_DMan                               0.000318
Lavadude     

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())